In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

In [6]:
df=pd.read_csv('final_data_new.csv')
X = df.drop(columns=['severity_class_1.0', 'severity_class_2.0', 'severity_class_3.0', 
                       'severity_class_4.0', 'severity_class_5.0', 'severity_class_6.0',
                       'severity_class_7.0', 'severity_class_8.0', 'severity_class_9.0','ev_id'])

y = df[['severity_class_1.0', 'severity_class_2.0', 'severity_class_3.0', 
          'severity_class_4.0', 'severity_class_5.0', 'severity_class_6.0',
          'severity_class_7.0', 'severity_class_8.0', 'severity_class_9.0']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

C:\Users\Tanushree\AppData\Local\Temp\ipykernel_18792\1890224880.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('final_data_new.csv')


RandomForestClassifier(random_state=42)

In [8]:
weights = {
    1: 1/(y.sum()[0]/len(y)),
    2: 1/(y.sum()[1]/len(y)),
    3: 1/(y.sum()[2]/len(y)),
    4: 1/(y.sum()[3]/len(y)),
    5: 1/(y.sum()[4]/len(y)),
    6: 1/(y.sum()[5]/len(y)),
    7: 1/(y.sum()[6]/len(y)),
    8: 1/(y.sum()[7]/len(y)),
    9: 1/(y.sum()[8]/len(y)),
    }

C:\Users\Tanushree\AppData\Local\Temp\ipykernel_18792\2998778069.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  1: 1/(y.sum()[0]/len(y)),
C:\Users\Tanushree\AppData\Local\Temp\ipykernel_18792\2998778069.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  2: 1/(y.sum()[1]/len(y)),
C:\Users\Tanushree\AppData\Local\Temp\ipykernel_18792\2998778069.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  3: 1/(y.sum()[2]/len(y)),
C:\User

In [9]:
y_single_column = y.idxmax(axis=1).str.extract('(\d+)').astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y_single_column, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, 
                                  random_state=42, 
                                  class_weight=weights)
rf_model.fit(X_train, y_train)


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight={1: 30.64146341463415, 2: 125.0049751243781,
                                     3: 220.4035087719298,
                                     4: 1.7438922820655192,
                                     5: 5.665388951521984,
                                     6: 10.710144927536232,
                                     7: 13.336518046709129,
                                     8: 43.171821305841924,
                                     9: 74.77976190476191},
                       random_state=42)

In [10]:
y_pred = rf_model.predict(X_test)
balance = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balance}")

Balanced Accuracy: 0.12973215706293612


In [11]:
from sklearn.model_selection import KFold, GridSearchCV

In [15]:
k = 5
fold_size = len(X) // k
for i in range(k):
    x_test2 = X.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([X.iloc[:i * fold_size], X.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [18]:
rf = RandomForestClassifier(
    n_estimators=100,       # Number of trees in the forest
    criterion='gini',       # Criterion: 'gini' (default) or 'entropy'
    max_depth=None,         # Maximum depth of the trees. None means no limit.
    min_samples_split=2,    # Minimum samples required to split an internal node
    min_samples_leaf=1,     # Minimum samples required to be at a leaf node
)
rf.fit(x_train2, y_train2)
y_pred2 = rf.predict(x_test2)

In [ ]:
cv = KFold(n_splits = 5, shuffle=False)  ### Specifies the k=5, number of splits in the cross-validation procedure
rf = RandomForestClassifier() 

params = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'bootstrap': [True, False]
}
grid_search = GridSearchCV(
    estimator=rf,       # Model
    param_grid=params,      # Parameter Grid: 
    scoring = 'balanced_accuracy',
    cv=cv,
    verbose=2,              # Print progression: 1 = very brief, 2 = detailed, 0 = none
    n_jobs=4,               # Number of processors to use from my CPU
)

grid_search.fit(X, y)